In [8]:
import cv2
import numpy as np
import os
import scipy.io as sio
from extract_random import extract_random
from cvpr_compare import cvpr_compare
from random import randint
from matplotlib import pyplot as plt


In [9]:
def extract_color_descriptor(img):
    # Compute the average red, green, and blue values as a basic color descriptor
    R = np.mean(img[:, :, 2])  # Note: OpenCV uses BGR format
    G = np.mean(img[:, :, 1])
    B = np.mean(img[:, :, 0])
    return np.array([R, G, B])

In [10]:
DATASET_FOLDER = "MSRC_ObjCategImageDatabase_v2"
OUT_FOLDER = "descriptors"
OUT_SUBFOLDER = 'globalRGBhisto'

if not os.path.exists(OUT_FOLDER):
    os.makedirs(OUT_FOLDER)

# Extract Image Descriptors from the Dataset
os.makedirs(os.path.join(OUT_FOLDER, OUT_SUBFOLDER), exist_ok=True)

In [11]:
for filename in os.listdir(os.path.join(DATASET_FOLDER, 'Images')):
    if filename.endswith(".bmp"):
        # print(f"Processing file {filename}")
        img_path = os.path.join(DATASET_FOLDER, 'Images', filename)
        img = cv2.imread(img_path).astype(np.float64) / 255.0  # Normalize the image
        fout = os.path.join(OUT_FOLDER, OUT_SUBFOLDER, filename).replace('.bmp', '.mat')
        # Call extractRandom (or another feature extraction function) to get the descriptor
        F = extract_random(img)        
        # Save the descriptor to a .mat file
        sio.savemat(fout, {'F': F})

In [12]:
DESCRIPTOR_FOLDER = 'descriptors'
DESCRIPTOR_SUBFOLDER = 'globalRGBhisto'

# Load all descriptors
ALLFEAT = []
ALLFILES = []
for filename in os.listdir(os.path.join(DESCRIPTOR_FOLDER, DESCRIPTOR_SUBFOLDER)):
    if filename.endswith('.mat'):
        descriptor_path = os.path.join(DESCRIPTOR_FOLDER, DESCRIPTOR_SUBFOLDER, filename)
        img_path = os.path.join(DATASET_FOLDER, 'Images', filename.replace('.mat', '.bmp'))
        descriptor_data = sio.loadmat(descriptor_path)
        ALLFILES.append(img_path)
        ALLFEAT.append(descriptor_data['F'][0])  # Assuming F is a 1D array
# Convert ALLFEAT to a numpy array
ALLFEAT = np.array(ALLFEAT)

# Pick a random image as the query
NIMG = ALLFEAT.shape[0]
queryimg = randint(0, NIMG - 1)

# Compute the distance between the query and all other descriptors
dst = []
query = ALLFEAT[queryimg]
for i in range(NIMG):
    candidate = ALLFEAT[i]
    distance = cvpr_compare(query, candidate)
    dst.append((distance, i))

# Sort the distances
dst.sort(key=lambda x: x[0])

In [17]:
# Show the top 15 results
SHOW = 15
images = [] 
target_height = 100

for i in range(min(SHOW, len(dst))):
    img_path = ALLFILES[dst[i][1]]
    img = cv2.imread(img_path)
    
    if img is None:
        print(f"Failed to load image at {img_path}")
        continue
    
    # Calculate the new width to maintain the aspect ratio
    aspect_ratio = img.shape[1] / img.shape[0]
    new_width = int(target_height * aspect_ratio)
    
    # Resize the image to the target height while maintaining the aspect ratio
    img = cv2.resize(img, (new_width, target_height))
    images.append(img)

# Concatenate images horizontally
if images:
    concatenated_image = np.hstack(images)
    cv2.imshow("Results", concatenated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
else:
    print("No images to display.")